<a href="https://colab.research.google.com/github/khizon/NBA_points/blob/main/NBA_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using NBA's Stats API to collect shot data**

In [77]:
!pip install nba_api --q

In [78]:
# Import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import time

plt.style.use('seaborn-darkgrid')

In [79]:
# Load teams file
teams = pd.DataFrame(json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json').text))
# Load players file
players = pd.DataFrame(json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/players.json').text))

In [80]:
# Get team ID based on team name
def get_team_id(team_abbreviation):
  try:
    return teams.loc[teams.abbreviation==team_abbreviation, 'teamId'].values[0]
  except:
    print(f'{team_abbreviation} is not an NBA Team')
    return -1
  
# Get player ID based on player name
def get_player_id(first, last):
  try:
    return players.loc[(players.firstName == first) & (players.lastName == last), 'playerId'].values[0]
  except:
    print(f'{first} {last} is not an NBA Player')
    return -1

# JSON to Df
def json_to_df(shot_json):
  # Load data into a Python dictionary
  shot_data = json.loads(shot_json.get_json())
  # Get the relevant data from our dictionary
  relevant_data = shot_data['resultSets'][0]
  # Get the headers and row data
  headers = relevant_data['headers']
  rows = relevant_data['rowSet']
  # Create pandas DataFrame
  df = pd.DataFrame(rows)
  df.columns = headers
  return df

In [81]:
np.random.randint(10)

6

In [82]:
season = 2003
df = pd.DataFrame()
for season in range(2003, 2023):
  if (season < 2010) | ((season >= 2014) & (season < 2018)):
    team = 'CLE'
  elif ((season>=2010) & (season<2014)):
    team = 'MIA'
  else:
    team = 'LAL'
  f_season = f'{season}-{(season+1)%100:02d}'
  print(f'{f_season} {team}')
  # Create JSON request
  shot_json = shotchartdetail.ShotChartDetail(
              team_id = get_team_id(team),
              player_id = get_player_id('LeBron', 'James'),
              context_measure_simple = 'PTS',
              season_nullable = f_season,
              season_type_all_star = 'Regular Season',
            )
  temp = json_to_df(shot_json)
  temp['SEASON'] = f_season

  df = df.append(temp)
  time.sleep(np.random.randint(10)) # Added some delay between API calls just in case.

df.shape

2003-04 CLE
2004-05 CLE
2005-06 CLE
2006-07 CLE
2007-08 CLE
2008-09 CLE
2009-10 CLE
2010-11 MIA
2011-12 MIA
2012-13 MIA
2013-14 MIA
2014-15 CLE
2015-16 CLE
2016-17 CLE
2017-18 CLE
2018-19 LAL
2019-20 LAL
2020-21 LAL
2021-22 LAL
2022-23 LAL


(14053, 25)

In [83]:
df = df.reset_index()
df = df.drop(['index'], axis=1)
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/lebron_shots.csv')

In [84]:
df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON
0,Shot Chart Detail,0020300014,20,2544,LeBron James,1610612739,Cleveland Cavaliers,1,8,56,...,8-16 ft.,15,158,0,1,1,20031029,SAC,CLE,2003-04
1,Shot Chart Detail,0020300014,28,2544,LeBron James,1610612739,Cleveland Cavaliers,1,7,44,...,8-16 ft.,13,-131,2,1,1,20031029,SAC,CLE,2003-04
2,Shot Chart Detail,0020300014,35,2544,LeBron James,1610612739,Cleveland Cavaliers,1,7,1,...,16-24 ft.,16,163,5,1,1,20031029,SAC,CLE,2003-04
3,Shot Chart Detail,0020300014,54,2544,LeBron James,1610612739,Cleveland Cavaliers,1,4,48,...,Less Than 8 ft.,5,54,22,1,1,20031029,SAC,CLE,2003-04
4,Shot Chart Detail,0020300014,67,2544,LeBron James,1610612739,Cleveland Cavaliers,1,3,3,...,Less Than 8 ft.,0,0,0,1,1,20031029,SAC,CLE,2003-04


In [85]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/lebron_shots.csv', index_col=0)
df.tail()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON
14048,Shot Chart Detail,22200821,426,2544,LeBron James,1610612747,Los Angeles Lakers,3,5,11,...,Less Than 8 ft.,3,18,26,1,1,20230207,LAL,OKC,2022-23
14049,Shot Chart Detail,22200821,476,2544,LeBron James,1610612747,Los Angeles Lakers,3,2,0,...,Less Than 8 ft.,2,12,25,1,1,20230207,LAL,OKC,2022-23
14050,Shot Chart Detail,22200821,486,2544,LeBron James,1610612747,Los Angeles Lakers,3,1,6,...,Less Than 8 ft.,2,-2,28,1,1,20230207,LAL,OKC,2022-23
14051,Shot Chart Detail,22200821,501,2544,LeBron James,1610612747,Los Angeles Lakers,3,0,10,...,8-16 ft.,13,-48,129,1,1,20230207,LAL,OKC,2022-23
14052,Shot Chart Detail,22200821,640,2544,LeBron James,1610612747,Los Angeles Lakers,4,1,51,...,Less Than 8 ft.,1,0,16,1,1,20230207,LAL,OKC,2022-23
